In [52]:
#This file aims to produce a real case scenario of an incident report
#The report will be wrote into a json file

In [77]:
import json
import random
import numpy as np
from Incident import Incident
from docplex.mp.model import Model
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
import dwave

In [54]:
#First of all, I've to instanciate dictionaries of controls and incidents
controls = {
    #control_key : [control_name, estimated_time]
    1 : ["Information Backup", 6],
    2 : ["Capacity Management", 6],
    3 : ["Off-Site Equipment Security", 24],
    4 : ["Physical Entry Controls", 8],
    5 : ["Restriction Of Access To Information", 40],
    6 : ["Management Of Secret Information Authentication", 40],
    7 : ["Equipment Maintenance", 24],
    8 : ["Capacity Management", 8],
    9 : ["Clock Synchronization", 2],
    10 : ["Information Backups", 6],
    11 : ["Supply Facilities", 72],
    12 : ["Network Controls", 8]
}

In [55]:
threats = {
    #threat_key : [threat_name, control_key1, control_key2, ...]
    1 : ["Denial Of Service", 1, 2],
    2 : ["Theft", 3],
    3 : ["Social Engineering", 4, 6],
    4 : ["Monitoring Errors", 5, 6],
    5 : ["Failure Of Physical Or Logical Origin", 7],
    6 : ["System Crashes Due To Resource Exhaustion", 8],
    7 : ["Abuse Of Access Privileges", 5],
    8 : ["Configuration Errors", 9],
    9 : ["Maintenance Errors", 10],
    10 : ["Inadequate Temperature Or Humidity Conditions", 11],
    11 : ["Failure Of Communication Services", 12]
}

In [56]:
penality = 73

In [57]:
#Now I've to make the dataset (of 14 incidents)
incidents = []
for _ in range(14):
    threat_key = random.randint(1,11)
    threat_list = threats.get(threat_key)
    #if the list contains just the threat name, we have to choose another threat
    while(len(threat_list) < 2):
        threat_key = random.randint(1,11)
        threat_list = threats.get(threat_key)
    threat_name = threat_list[0]
    control_key = threat_list.pop(random.randint(1,len(threat_list)-1))
    control_list = controls[control_key]
    control_name = control_list[0]
    estimated_time = control_list[1]
    incident = Incident(threat_key, threat_name, control_key,control_name, estimated_time)
    incidents.append(incident)

for i in range(len(incidents)):
    print(incidents[i])

threat_id: 5, threat_name: Failure Of Physical Or Logical Origin, control_id: 7, control_name: Equipment Maintenance, estimated_time: 24
threat_id: 11, threat_name: Failure Of Communication Services, control_id: 12, control_name: Network Controls, estimated_time: 8
threat_id: 2, threat_name: Theft, control_id: 3, control_name: Off-Site Equipment Security, estimated_time: 24
threat_id: 1, threat_name: Denial Of Service, control_id: 2, control_name: Capacity Management, estimated_time: 6
threat_id: 7, threat_name: Abuse Of Access Privileges, control_id: 5, control_name: Restriction Of Access To Information, estimated_time: 40
threat_id: 9, threat_name: Maintenance Errors, control_id: 10, control_name: Information Backups, estimated_time: 6
threat_id: 6, threat_name: System Crashes Due To Resource Exhaustion, control_id: 8, control_name: Capacity Management, estimated_time: 8
threat_id: 1, threat_name: Denial Of Service, control_id: 1, control_name: Information Backup, estimated_time: 6
t

In [66]:
def create_QUBO_problem(linear_terms,quadratic_terms):
    model = Model()

    #i nomi delle var lineare saranno xt1,...,xtn. Rappresentano il tempo stimato per ogni incidente in esame
    x = [model.binary_var("xt%s" % (i+1)) for i in range(len(quadratic_terms))]

    #togli la penalità al termine xti
    objective = (- penality) + model.sum([linear_terms[i] * x[i] for i in range(len(linear_terms))])
    #aggiungi il doppio della penalità agli incidenti con controlli comuni
    objective += (2 * penality) + model.sum(
        [quadratic_terms[i, j] * x[i] * x[j] for i in range(len(linear_terms)) for j in range(len(quadratic_terms))]
    )

    #vogliamo minimizzare l'hamiltoniana
    model.minimize(objective)

    #convertiamo il modello in un modello QUBO
    qubo = from_docplex_mp(model)

    cost = model.sum(x)
    model.add_constraint(cost == 3)

    return qubo

In [67]:
linear_terms = np.array([0,0,8,40,24,8,24,0,0,72,16,8])
quadratic_terms = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,73,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0]
])

In [68]:
qubo = create_QUBO_problem(linear_terms,quadratic_terms)
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model12

Minimize
 obj: 8 xt3 + 40 xt4 + 24 xt5 + 8 xt6 + 24 xt7 + 72 xt10 + 16 xt11 + 8 xt12 + [
      146 xt4*xt11 ]/2 + 73
Subject To

Bounds
 0 <= xt1 <= 1
 0 <= xt2 <= 1
 0 <= xt3 <= 1
 0 <= xt4 <= 1
 0 <= xt5 <= 1
 0 <= xt6 <= 1
 0 <= xt7 <= 1
 0 <= xt8 <= 1
 0 <= xt9 <= 1
 0 <= xt10 <= 1
 0 <= xt11 <= 1
 0 <= xt12 <= 1

Binaries
 xt1 xt2 xt3 xt4 xt5 xt6 xt7 xt8 xt9 xt10 xt11 xt12
End